## Table of section and pillar names
|    | News | Opinion | Sport |  Culture  |  Life style
| --- | --- | --- | --- | --- | --- |
| Section names |1.	Coronavirus |11.The Guardian view | 15.	Cricket | 22.Books | 30.Fashion |
|  |1.	Coronavirus | 12.Columnists |16.Rugby union | 23.Music | 31.Food |
|  |2.	World news | 13.Opinion videos |17.	Tennis | 24.TV&Culture | 32.Recipes |
|  |3.	UK news | 14.Letters |18.Cycling | 25.Art&design| 33.Love&sex |
|  |4.	Environment |  | 19.F1 | 26.Film | 34.Health&fitness |
|  |5.	Science |  | 20.Golf |27.Games| 35.Home&garden  |
|  |6.	Global development |  | 21.US sports | 28.Classical | 36.Women |
|  |7.	Football |  |  | 29.Stage | 37.Men |
|  |8.	Tech | |  |  | 38.Family |
|  |9.	Business |  |  |  | 39.Travel |
|  |10.	Obituaries|  |  |  | 40.Money |

## Function 1: Collect the news content with all information
**input: fromdate, enddate, target word query and output file name**  
**output: news content information in json format and saved as txt file**

In [1]:
import json
import requests
import html2text,pandas as pd
import csv,xlsxwriter
from bs4 import BeautifulSoup
from openpyxl import load_workbook
import re
import schedule
import time
import xlwt


In [17]:
# Refer the doc - https://open-platform.theguardian.com/documentation/

def ori_guardian(fromdate, todate,q,file):
    API_KEY = "6ad1613f-4f40-4984-b0a9-f288f42ec958"  #get your api-key from https://open-platform.theguardian.com/access/
    
    API_ENDPOINT = 'http://content.guardianapis.com/search' 
    '''
    fromdate: the starting date of the news collection (date of news is based on the publication date)
    
    todate: the end date of the news collection (same as the above)
    
    q: the query you want to collect. 
    For example, if you want to collect news contain keywords: economy, immigration and us, 
    your q can be: “economy AND immigration AND us”. It also supports OR and AND NOT.
    The details of query you can make can refer this link: https://open-platform.theguardian.
    
    file: function will output a file contains news information, 
    you can customize the file namecom/documentation/ 

    '''
    
    my_params = {
        'q':"",#q is query
        'from-date': "", 
        'to-date': "", 
        'order-by':"oldest", 
        'show-fields': "all",
        'page-size':10 ,
        'api-key': API_KEY,
        'lang':"",
    }
    
    my_params['from-date'] = fromdate
    my_params['to-date'] = todate
    my_params['q'] = q
    my_params['lang'] = "en"
    
    
    startrow=None 
    current_page = 1
    total_pages = 1
    url_list = []
    
    while current_page <= total_pages:
        print("...page", current_page)
        my_params['page'] = current_page
        resp = requests.get(API_ENDPOINT, my_params)
        print(resp.url)
        print("\n")
        data = resp.json()
        current_page += 1
        total_pages=data['response']['pages']
        url_list.append(data)
    file = file
    with open(str(file)+'guardian.txt', 'w') as filehandle:
        json.dump(url_list, filehandle)

### Function 1 example

In [18]:
 ori_guardian('2019-01-01', '2019-01-31','immigration','try')

...page 1
http://content.guardianapis.com/search?q=immigration&from-date=2019-01-01&to-date=2019-01-31&order-by=oldest&show-fields=all&page-size=10&api-key=6ad1613f-4f40-4984-b0a9-f288f42ec958&lang=en&page=1


...page 2
http://content.guardianapis.com/search?q=immigration&from-date=2019-01-01&to-date=2019-01-31&order-by=oldest&show-fields=all&page-size=10&api-key=6ad1613f-4f40-4984-b0a9-f288f42ec958&lang=en&page=2


...page 3
http://content.guardianapis.com/search?q=immigration&from-date=2019-01-01&to-date=2019-01-31&order-by=oldest&show-fields=all&page-size=10&api-key=6ad1613f-4f40-4984-b0a9-f288f42ec958&lang=en&page=3


...page 4
http://content.guardianapis.com/search?q=immigration&from-date=2019-01-01&to-date=2019-01-31&order-by=oldest&show-fields=all&page-size=10&api-key=6ad1613f-4f40-4984-b0a9-f288f42ec958&lang=en&page=4


...page 5
http://content.guardianapis.com/search?q=immigration&from-date=2019-01-01&to-date=2019-01-31&order-by=oldest&show-fields=all&page-size=10&api-key=6ad1

## Function 2: collect news content witn main informaton
**input: fromdate, enddate, target word query and output file name**  
**output: newsid, section name, pilar name, content, webURL, query, title and published date and saved as excel file**

In [25]:
def guardian(fromdate, todate,q,file):
    API_KEY = "6ad1613f-4f40-4984-b0a9-f288f42ec958"  #get your api-key from https://open-platform.theguardian.com/access/
    
    API_ENDPOINT = 'http://content.guardianapis.com/search' 
    
    
    my_params = {
        'q':"",#q is query
        'from-date': "", 
        'to-date': "", 
        'order-by':"oldest", 
        'show-fields': "all",
        'page-size':10 ,
        'api-key': API_KEY,
        'lang':"",
    }
    '''
    fromdate: the starting date of the news collection (date of news is based on the publication date)
    
    todate: the end date of the news collection (same as the above)
    
    q: the query you want to collect. 
    For example, if you want to collect news contain keywords: economy, immigration and us, 
    your q can be: “economy AND immigration AND us”. It also supports OR and AND NOT.
    The details of query you can make can refer this link: https://open-platform.theguardian.
    
    file: function will output a file contains news information, 
    you can customize the file namecom/documentation/ 

    '''
    
    my_params['from-date'] = fromdate
    my_params['to-date'] = todate
    my_params['q'] = q
    my_params['lang'] = "en"
    
    id_list=[]
    pillarname=[]
    web=[]
    body_list=[] 
    title_list=[]
    date_list=[]
    section=[]
    topic=[] 
    apiurl=[]
    startrow=None 
    current_page = 1
    total_pages = 1
    
    while current_page <= total_pages:
                print("...page", current_page)
                my_params['page'] = current_page
                resp = requests.get(API_ENDPOINT, my_params)
                print(resp.url)
                print("\n")
                data = resp.json()
                current_page += 1
                total_pages=data['response']['pages']

                results=len(data["response"]["results"])
                
                for i in range(0,results):
                    pid=data["response"]["results"][i]["id"]
                    pp=html2text.html2text(pid) 
                    id_list.append(pp)
                    
                    pn=data["response"]["results"][i]["pillarName"]
                    ppnn=html2text.html2text(pn)
                    pillarname.append(ppnn)
                    
                    b=data["response"]["results"][i]["fields"]["bodyText"]
                    b.replace("\n","")
                    bb=html2text.html2text(b) 
                    bb = re.sub(r"\(https\:\S+|\s+\)", "", bb)
                    body_list.append(bb)
                    
                    t=data["response"]["results"][i]["webTitle"]
                    t.replace("\n","")
                    tt=html2text.html2text(t) 
                    tt = re.sub(r"\(https\:\S+|\s+\)", "", tt)
                    title_list.append(tt)
                    
                    s=data["response"]["results"][i]["sectionName"]
                    s.replace("\n","")
                    ss=html2text.html2text(s) 
                    ss = re.sub(r"\(https\:\S+|\s+\)", "", ss)
                    section.append(ss)
                    
                    
                    dt=data["response"]["results"][i]["webPublicationDate"]
                    dt.replace("\n","")
                    ddtt=html2text.html2text(dt) 
                    ddtt = re.sub(r"\(https\:\S+|\s+\)", "", ddtt)
                    date_list.append(ddtt)
                    
                    wb=data["response"]["results"][i]["webUrl"]
                    wwbb=html2text.html2text(wb) 
                    web.append(wwbb)
                    
                    ap=data["response"]["results"][i]["apiUrl"]
                    aapp=html2text.html2text(ap)
                    apiurl.append(aapp)
                    
                    topic.append(my_params.get("q")) 
                    
                list_of_tuples = list(zip(id_list, section, pillarname, body_list, web, apiurl, topic, title_list, date_list))  
                
                df = pd.DataFrame(list_of_tuples,index=None, columns=['id','section', 'pillarname', 'content','webUrl', 'apiurl',
                                                                      'query', 'title', 'date'])
                
                filename = r"guardian.xls"
                file=file
                df.to_excel(str(file)+"guardian.xlsx",index=False,header=True) # it overwrites the excel file 
                return df #can delete it, just want to show to you what's the datarframe look like
                """
                # it appends the dataframe df to the excel
                writer = pd.ExcelWriter(filename, engine='openpyxl')
                writer.book = load_workbook(filename)
                if  "Sheet1" in writer.book.sheetnames:
                    startrow = writer.book["Sheet1"].max_row
                    
                writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
                
                if startrow is None:
                    startrow = 0
                df.to_excel(writer, "Sheet1", startrow=startrow,index=False,header=False)
                writer.save()
                writer.close()
    """           

### Function 2 example

In [26]:
guardian('2019-01-01', '2019-01-31', 'airlines','airline_201901')

...page 1
http://content.guardianapis.com/search?q=airlines&from-date=2019-01-01&to-date=2019-01-31&order-by=oldest&show-fields=all&page-size=10&api-key=6ad1613f-4f40-4984-b0a9-f288f42ec958&lang=en&page=1




,id,section,pillarname,content,webUrl,apiurl,query,title,date
0,business/2019/jan/02/bestjet-collapse-leaves-a...,Business\n\n,News\n\n,Travellers who booked flights through Australi...,https://www.theguardian.com/business/2019/jan/...,https://content.guardianapis.com/business/2019...,airlines,Bestjet collapse leaves angry customers thousa...,2019-01-02T04:47:30Z\n\n
1,business/2019/jan/02/cathay-pacific-blunder-of...,Business\n\n,News\n\n,It was the New Year’s Eve travel sale that app...,https://www.theguardian.com/business/2019/jan/...,https://content.guardianapis.com/business/2019...,airlines,"Cathay Pacific error sees $16,000 flights sold...",2019-01-02T13:13:04Z\n\n
2,business/2019/jan/03/ryanair-secures-guarantee...,Business\n\n,News\n\n,Ryanair has been granted a UK air operating ce...,https://www.theguardian.com/business/2019/jan/...,https://content.guardianapis.com/business/2019...,airlines,Ryanair secures guarantee from CAA for post-Br...,2019-01-03T17:13:32Z\n\n
3,business/2019/jan/04/budget-airlines-industry-...,Business\n\n,News\n\n,"Herb Kelleher, the co-founder of Southwest Air...",https://www.theguardian.com/business/2019/jan/...,https://content.guardianapis.com/business/2019...,airlines,"Herb Kelleher, pioneer of low-cost air travel,...",2019-01-04T13:20:14Z\n\n
4,business/2019/jan/05/strike-hit-ryanair-ranked...,Business\n\n,News\n\n,Ryanair has been rated as the worst airline fo...,https://www.theguardian.com/business/2019/jan/...,https://content.guardianapis.com/business/2019...,airlines,Ryanair ranked 'worst airline' for sixth year ...,2019-01-05T10:44:47Z\n\n
5,commentisfree/2019/jan/06/managed-no-deal-brex...,Opinion\n\n,Opinion\n\n,"Here we go again. As we return, haggard and hu...",https://www.theguardian.com/commentisfree/2019...,https://content.guardianapis.com/commentisfree...,airlines,‘Managed no deal’? That’s just more Brexit sna...,2019-01-06T13:17:42Z\n\n
6,football/2019/jan/08/sunderland-newcastle-not-...,Football\n\n,Sport\n\n,The big news on Wearside on Monday revolved ar...,https://www.theguardian.com/football/2019/jan/...,https://content.guardianapis.com/football/2019...,airlines,Sunderland meet Newcastle – but this is not th...,2019-01-08T09:30:05Z\n\n
7,commentisfree/2019/jan/09/free-societies-russi...,Opinion\n\n,Opinion\n\n,How do you respond when a principle that forms...,https://www.theguardian.com/commentisfree/2019...,https://content.guardianapis.com/commentisfree...,airlines,When free societies copy Russian media tactics...,2019-01-09T16:00:17Z\n\n
8,travel/2019/jan/13/25-holidays-retreats-to-reb...,Travel\n\n,Lifestyle\n\n,"1\. Lošinj, Croatia Eva Wiseman visits a tradi...",https://www.theguardian.com/travel/2019/jan/13...,https://content.guardianapis.com/travel/2019/j...,airlines,25 holidays to reboot your life\n\n,2019-01-13T07:00:07Z\n\n
9,politics/2019/jan/14/britons-living-in-eu-new-...,Politics\n\n,News\n\n,British people living in the EU may have to ta...,https://www.theguardian.com/politics/2019/jan/...,https://content.guardianapis.com/politics/2019...,airlines,Britons in EU may have to retake driving tests...,2019-01-14T16:38:54Z\n\n


## Function3(guardian_tag)
**input: starting date, end date, tag, file name**   
**output: keyword of the news, website title, webURL, tag(input one) and saved as excel file**

In [23]:
# Refer the doc - https://open-platform.theguardian.com/documentation/

def guardian_tag(fromdate, todate,tag,file):
    API_KEY = "6ad1613f-4f40-4984-b0a9-f288f42ec958"  #get your api-key from https://open-platform.theguardian.com/access/
    
    API_ENDPOINT = 'http://content.guardianapis.com/tags' 
    
#https://content.guardianapis.com/search?tag=environment/recycling&api-key=test    
    my_params = {
        'tag':"",#tag is query
        'from-date': "", 
        'to-date': "", 
        'order-by':"oldest", 
        'show-fields': "all",
        'page-size':10 ,
        'api-key': API_KEY,
        'lang':"",
    }
    '''
    fromdate: the starting date of the news collection (date of news is based on the publication date)
    
    todate: the end date of the news collection (same as the above)
    
    q: the query you want to collect. 
    For example, if you want to collect news contain keywords: economy, immigration and us, 
    your q can be: “economy AND immigration AND us”. It also supports OR and AND NOT.
    The details of query you can make can refer this link: https://open-platform.theguardian.
    
    file: function will output a file contains news information, 
    you can customize the file namecom/documentation/ 

    '''
    my_params['from-date'] = fromdate
    my_params['to-date'] = todate
    my_params['tag'] = tag
    my_params['lang'] = "en"
    
    keyword=[] 
    web=[]
    topic=[] 
    title=[]
    apiurl=[]
    startrow=None 
    current_page = 1
    total_pages = 1
    
    while current_page <= total_pages:
                print("...page", current_page)
                my_params['page'] = current_page
                resp = requests.get(API_ENDPOINT, my_params)
                print(resp.url)
                print("\n")
                data = resp.json()
                current_page += 1
                total_pages=data['response']['pages']

                results=len(data["response"]["results"])
                
                for i in range(0,results):
                    k=data["response"]["results"][i]["id"]
                    kk=html2text.html2text(k) 
                    keyword.append(kk)
                    
                   
                    wb=data["response"]["results"][i]["webUrl"]
                    wwbb=html2text.html2text(wb) 
                    web.append(wwbb)
                    
                    ap=data["response"]["results"][i]["apiUrl"]
                    aapp=html2text.html2text(ap)
                    apiurl.append(aapp)
                    
                    t=data["response"]["results"][i]["webTitle"]
                    tt=html2text.html2text(t) 
                    title.append(tt)
                    
                    topic.append(my_params.get("tag")) 
                    
                list_of_tuples = list(zip(keyword,title, web, apiurl,topic))  
                
                df = pd.DataFrame(list_of_tuples,index=None, columns=['keyword','webtitle','web', 'apiurl','query'])
                
                filename = r"guardian.xls"
                file=file
                df.to_excel(str(file)+"guardian.xlsx",index=False,header=True) # it overwrites the excel file 
                return df #can delete it, just show what's dataframe like
                """
                # it appends the dataframe df to the excel
                writer = pd.ExcelWriter(filename, engine='openpyxl')
                writer.book = load_workbook(filename)
                if  "Sheet1" in writer.book.sheetnames:
                    startrow = writer.book["Sheet1"].max_row
                    
                writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
                
                if startrow is None:
                    startrow = 0
                df.to_excel(writer, "Sheet1", startrow=startrow,index=False,header=False)
                writer.save()
                writer.close()
    """           

### Function 3 example

In [24]:
guardian_tag('2019-01-01', '2019-01-31', 'economy','eco_201901')

...page 1
http://content.guardianapis.com/tags?tag=economy&from-date=2019-01-01&to-date=2019-01-31&order-by=oldest&show-fields=all&page-size=10&api-key=6ad1613f-4f40-4984-b0a9-f288f42ec958&lang=en&page=1




,keyword,webtitle,web,apiurl,query
0,100-teachers/100-teachers\n\n,"100 teachers, 100 passions, 100 ways to shape ...",https://www.theguardian.com/100-teachers/100-t...,https://content.guardianapis.com/100-teachers/...,economy
1,2019-family-gift-guide/2019-family-gift-guide\n\n,2019 family gift guide\n\n,https://www.theguardian.com/2019-family-gift-g...,https://content.guardianapis.com/2019-family-g...,economy
2,20th-century-fox-the-shape-of-water/20th-centu...,20th Century Fox: The Shape of Water\n\n,https://www.theguardian.com/20th-century-fox-t...,https://content.guardianapis.com/20th-century-...,economy
3,a-different-christmas/series/gifting-reimagine...,Gifting reimagined\n\n,https://www.theguardian.com/a-different-christ...,https://content.guardianapis.com/a-different-c...,economy
4,a-different-christmas/series/on-the-climate-fr...,On the climate frontline\n\n,https://www.theguardian.com/a-different-christ...,https://content.guardianapis.com/a-different-c...,economy
5,a-place-called-home/a-place-called-home\n\n,A place called home\n\n,https://www.theguardian.com/a-place-called-hom...,https://content.guardianapis.com/a-place-calle...,economy
6,a-shot-of-sustainability/a-shot-of-sustainabil...,a shot of sustainability\n\n,https://www.theguardian.com/a-shot-of-sustaina...,https://content.guardianapis.com/a-shot-of-sus...,economy
7,aberdeen-grampian/aberdeen-grampian\n\n,Aberdeen-Grampian\n\n,https://www.theguardian.com/aberdeen-grampian/...,https://content.guardianapis.com/aberdeen-gram...,economy
8,about/about\n\n,About\n\n,https://www.theguardian.com/about/about\n\n,https://content.guardianapis.com/about/about\n\n,economy
9,about/test-tag--do-not-use-this-\n\n,Test Tag (DO NOT USE THIS)\n\n,https://www.theguardian.com/about/test-tag--do...,https://content.guardianapis.com/about/test-ta...,economy
